In [1]:
# gensim modules
from gensim import utils
from gensim.models.doc2vec import LabeledSentence
from gensim.models import Doc2Vec
from gensim.models import Word2Vec
import gensim
import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
from konlpy.utils import pprint
import time
from konlpy.tag import Kkma, Twitter, Komoran
import csv
from scipy import spatial
import pyemd
from datetime import datetime
import pandas as pd
import quandl
import pickle

# numpy
import numpy as np

#정규식
import re

# random
from random import shuffle

C:\Users\rjsfu\Anaconda3\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [53]:
#label과 data 읽기
from os import listdir
from os.path import isfile, join

name = ['KB금융','LG생활건강', 'LG전자','LG화학','SK텔레콤','SK하이닉스', '기아차','네이버', '삼성SDI','삼성SDS','삼성생명','삼성전자','셀트리온','신한지주','아모레퍼시픽', '웅진코웨이','포스코', '한국가스공사','현대모비스','현대제철']
year = ['2008','2009','2010','2011','2012','2013','2014','2015','2016','2017']

data = []
for i in name:
    company_labels = []
    company_data = []
    for j in year:
        my_path = 'C:/Users/rjsfu/Dropbox/task/텍스트마이닝/data/사업보고서/'+i+'/'+j
        for k in listdir(my_path):
            if k.endswith('.txt'):
                with open(my_path + '/' + k, 'r', encoding = 'utf-8') as f2:
                    document = f2.read()
                    document = re.sub('[a-zA-Z]', '', document)
                    document = re.sub('[【】◆•◇ⓒ○→*■ㆍ\t\n\{\}\[\]\/?,;:|\)*~`!^\-_+<>@\#$%&\\\=\(\'\"]',
                                          '', document)
                    document = document.replace("\xa0", " ")
                    document = document[3:]
                    company_data.append(document)
                    f2.close()
    data.append(company_data)

In [59]:
#문서를 형태소 분리
tagger = Twitter()
tokens = []
tag = ['Noun', 'Advjective']
process_time= time.time()
for company_data in data:
    tokens_company = []
    for text in company_data:
        tagged = tagger.pos(text)
        for word, pos in tagged:
            if pos in tag:
                tokens_company.append(word)
    tokens.append(tokens_company)
process_time = time.time() - process_time
print('%.3f secs' % (process_time))

#한글자 지우기
remove_one = []
for i in tokens:
    company_remove_one = []
    for j in i:
        if len(j) == 1:
            pass
        else:
            company_remove_one.append(j)
    remove_one.append(company_remove_one)

2087.408 secs


In [60]:
#분리된 token pickle 파일로 저장하기
out_path = "C:/Users/rjsfu/Dropbox/task/텍스트마이닝/pickle data"

out_file = out_path + "/사업보고서_pos_data.pkl"
with open(out_file, 'wb') as f:
    pickle.dump(remove_one, f)

In [3]:
#pickle 파일 불러오기
with open('C:/Users/rjsfu/Dropbox/task/텍스트마이닝/pickle data/사업보고서_remove_one.pkl', 'rb') as f:
    remove_one_from_pkl = pickle.load(f)

In [31]:
#Word2Vec(CBOW)
process_time= time.time()
model_cbow_사업보고서 = gensim.models.Word2Vec(remove_one_from_pkl, size=100, window=2, min_count=5, workers=4, iter=50)
process_time = time.time() - process_time
print('%.3f secs' % (process_time))

# 학습이 완료 되면 필요없는 메모리를 unload 시킨다.
model_cbow_사업보고서.init_sims(replace=True)

model_name = 'word2vec_cbow_사업보고서'
# model_name = '300features_50minwords_20text'
model_cbow_사업보고서.save(model_name)

# check embedding result
print(model_cbow_사업보고서)

21.005 secs
Word2Vec(vocab=14276, size=100, alpha=0.025)


In [32]:
vocab = list(model_cbow_사업보고서.wv.vocab) 
embedding_words = model_cbow_사업보고서[vocab]

C:\Users\rjsfu\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [8]:
embedding_words[0]

array([-0.00796154,  0.12803733, -0.11547044,  0.05749838,  0.08860067,
        0.01592146, -0.01964619,  0.09962222,  0.03860348, -0.1234769 ,
       -0.00562428, -0.04935596,  0.07249469,  0.20893121,  0.13459644,
        0.05979065, -0.11171649,  0.10554021,  0.0019349 ,  0.21493824,
        0.07415015,  0.01761789,  0.03248944,  0.14705496, -0.02634456,
        0.00791652, -0.00212885, -0.14811015,  0.12052131,  0.0133042 ,
        0.16081066,  0.0863888 , -0.01344391, -0.10347986,  0.0669786 ,
        0.01336174, -0.11737291,  0.12385345,  0.11282435, -0.02247885,
        0.0029348 , -0.15065914,  0.13072127, -0.01629388,  0.22899726,
       -0.1469179 , -0.09405491, -0.14965346, -0.09501475, -0.03713531,
       -0.02030833, -0.0206362 , -0.05680463, -0.0651814 , -0.04032468,
       -0.04089382,  0.02320512,  0.04290888,  0.06831507, -0.07381016,
        0.12465953,  0.14560127,  0.00679597, -0.21757007,  0.16838145,
        0.14448921, -0.14250138,  0.07873455,  0.1409132 , -0.22

In [11]:
embedding_words[0]

array([-0.07068531,  0.05085431, -0.07768932,  0.02894349,  0.05006111,
       -0.00522722,  0.03601317, -0.00317973,  0.05254297, -0.18457933,
       -0.03034455, -0.09396613,  0.03509444,  0.23846099,  0.12026407,
        0.12616847, -0.19998468,  0.00166542, -0.00811057,  0.14855789,
        0.00713504,  0.08118285,  0.01178003,  0.09112039, -0.11443909,
       -0.0468833 , -0.0850499 , -0.09894614,  0.14910239, -0.01887487,
        0.13872673,  0.00871161, -0.09915053, -0.13172582,  0.05244549,
       -0.02693308, -0.13655172,  0.2019496 ,  0.1763572 , -0.05010791,
       -0.110985  , -0.00236284,  0.17807569,  0.04236151,  0.20892225,
       -0.00481202, -0.10043453, -0.09690817, -0.06917384, -0.02347186,
       -0.07418215, -0.11609393,  0.01317927, -0.00934981, -0.11817419,
       -0.04490226, -0.05493538, -0.09226307,  0.04369746, -0.08634626,
        0.10603034, -0.00035066, -0.03000004, -0.2055445 ,  0.06758034,
        0.12027048, -0.10443534,  0.05870871,  0.1444042 , -0.25

In [14]:
embedding_words[0]

array([ 0.0084443 ,  0.09343258, -0.03366252,  0.00085527, -0.00351295,
        0.00659117, -0.04195977, -0.06578787,  0.05328043, -0.10430244,
       -0.21568118,  0.08807699,  0.0916364 ,  0.18617176,  0.18294424,
        0.2225504 , -0.1761767 , -0.02780354,  0.00946041,  0.13948664,
        0.04825008,  0.00331038, -0.00628545,  0.0404609 , -0.14282252,
       -0.0798327 , -0.05296269, -0.08702714,  0.16299911, -0.0040556 ,
        0.14479025, -0.0763727 , -0.11469264, -0.13919969,  0.12056749,
       -0.02991412, -0.11429172,  0.1431371 ,  0.09371108, -0.08259164,
       -0.13750888,  0.04876134,  0.18810774,  0.06047216,  0.1250623 ,
       -0.05791865, -0.08822519, -0.03187174, -0.1852519 ,  0.00837848,
       -0.10391583, -0.10641146,  0.03601464,  0.04120228, -0.0977177 ,
       -0.03039605, -0.09941009, -0.0285903 ,  0.01529683, -0.13943909,
        0.15329   ,  0.01207265,  0.07237448, -0.15267384,  0.10625263,
        0.02346151, -0.11128362,  0.00614641,  0.17264636, -0.14

In [17]:
embedding_words[0]

array([ 0.04465323,  0.0318705 , -0.11526435, -0.08279957, -0.02085723,
        0.17914501, -0.14357994,  0.14769259, -0.06487294, -0.01469676,
       -0.14101994,  0.1359    , -0.04849598,  0.06960867, -0.12362091,
       -0.02765574,  0.01465781, -0.02263968, -0.12572935,  0.36451066,
        0.02273004, -0.00990091,  0.03719541, -0.04615543, -0.10699521,
       -0.05055238, -0.07995931,  0.03471436, -0.07991339,  0.12344667,
        0.17979462,  0.06325968,  0.02914447,  0.07502194, -0.01938559,
       -0.08147816, -0.17930982, -0.02447792, -0.19886187,  0.14437039,
        0.00292159,  0.0283832 , -0.09036138, -0.134076  ,  0.03565341,
       -0.0263809 , -0.0160129 , -0.11386777,  0.1012096 ,  0.13008423,
        0.05963612, -0.05145067,  0.00117266, -0.13717438,  0.0636894 ,
        0.08047656,  0.07509813,  0.00404469, -0.02254203, -0.0230038 ,
       -0.00568582, -0.00116999, -0.25674507,  0.05012523, -0.00775233,
       -0.19010945, -0.01943782, -0.0547296 , -0.07723097,  0.09

In [20]:
embedding_words[0]

array([ 2.74216831e-02,  6.88954070e-02, -2.39049308e-02, -1.99417900e-02,
        8.37056711e-03, -8.51967779e-04,  5.67405717e-04,  1.49860352e-01,
       -1.27329022e-01, -4.02867310e-02, -7.30031356e-02,  5.84851839e-02,
       -6.37217462e-02,  8.13129023e-02, -1.37740701e-01, -1.37935668e-01,
        1.00310108e-05,  4.67313491e-02, -1.24580093e-01,  3.30299467e-01,
       -6.83870986e-02, -6.57627657e-02,  2.78660785e-02,  1.83097739e-02,
       -6.67198841e-03,  7.44056180e-02, -4.98601235e-02, -1.35762310e-02,
       -1.51139244e-01,  1.60155118e-01,  1.75234973e-01,  6.16142042e-02,
        2.03041676e-02,  7.33561218e-02, -3.14734764e-02, -1.04703553e-01,
       -1.02655098e-01,  2.86239595e-03, -1.17573760e-01,  1.05101392e-01,
        4.75276187e-02, -9.96790007e-02, -1.42189622e-01, -1.00181535e-01,
        1.53169364e-01, -1.41115850e-02, -5.24295419e-02, -7.58041516e-02,
        1.30350143e-01,  1.31966323e-01,  8.71441439e-02,  4.82271202e-02,
       -1.17805116e-01, -

In [33]:
#centrality
from networkx.algorithms.centrality import degree_centrality
from sklearn.metrics.pairwise import rbf_kernel

vocab = list(model_cbow_사업보고서.wv.vocab) 
embedding_words = model_cbow_사업보고서[vocab]

affinity_matrix = rbf_kernel(embedding_words, gamma=1)

centrality = []
for i in range(len(affinity_matrix)):
    weight_sum = affinity_matrix[i].sum()
    centrality.append([i,weight_sum])
    
sroted_centrality = sorted(centrality, key = lambda centrality : centrality[1])[::-1]

centrality_index = []
for i in sroted_centrality:
    centrality_index.append(i[0])
    
sorted_by_centrality = []
for i in range(len(vocab)):
    sorted_by_centrality.append(vocab[centrality_index[i]])

C:\Users\rjsfu\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [34]:
#CSV 파일로 내보내기
import csv    
f = open('C:/Users/rjsfu/Dropbox/task/텍스트마이닝/관련 자료/centrality_word_사업보고서.csv', 'w', newline = '')
wr = csv.writer(f)
for i in sorted_by_centrality:
    wr.writerow([i])
f.close()

In [25]:
sorted_by_centrality

['의정부',
 '군산시',
 '성북구',
 '강북',
 '용산',
 '동두천시',
 '영천시',
 '희귀',
 '도봉',
 '관악',
 '무빙',
 '부곡',
 '파괴',
 '김포시',
 '포집',
 '서초',
 '떡잎',
 '목포',
 '곡물',
 '원통',
 '송염',
 '장형',
 '예대율',
 '부산진구',
 '여수시',
 '일드',
 '강서',
 '마포',
 '법정',
 '알레르기',
 '납입액',
 '과천',
 '거절',
 '사무실',
 '상단',
 '우레',
 '울트라',
 '채권양도',
 '금융채',
 '환전',
 '강남',
 '과일',
 '광명',
 '고성군',
 '음식',
 '통지',
 '성동구',
 '원주시',
 '사천시',
 '가산',
 '자세',
 '리기',
 '포천군',
 '유무',
 '용강',
 '한비',
 '노사',
 '파주시',
 '목포시',
 '여신전문금융업법',
 '구비',
 '농축액',
 '프로토타입',
 '이송',
 '미네랄',
 '무궁화',
 '색인',
 '평창',
 '야채',
 '영주',
 '종교',
 '웨어',
 '전라북도',
 '플루',
 '세법',
 '생기',
 '일본어',
 '포장재',
 '종이',
 '토코페롤',
 '약국',
 '거제',
 '비제',
 '순천시',
 '일상',
 '원본',
 '내츄럴',
 '생리',
 '송현동',
 '기분',
 '입방',
 '존중',
 '값임',
 '밸런싱',
 '도포',
 '딸기',
 '충청남도',
 '실의',
 '그램',
 '지시',
 '슬로건',
 '서라벌',
 '외피',
 '송내동',
 '염증',
 '도곡동',
 '통영시',
 '가루',
 '아바타',
 '발송',
 '리듬',
 '동물',
 '구로',
 '청구권',
 '페이트',
 '염료',
 '볼륨',
 '케익',
 '대구광역시',
 '가격결정',
 '조리',
 '경상북도',
 '전주시',
 '충청북도',
 '픽셀',
 '참빛',
 '갈란투스',
 '케미칼',
 '전라남도',
 '화제',
 '정충',
 '서초구',
 '